In [1]:
import os
import re
import subprocess

import mgitools.os_helpers as os_helpers

In [2]:
# !conda install -y -c conda-forge gsutil

In [3]:
upload_dir = '/home/estorrs/imaging/uploads/10072021/level2/'

## prepare to upload to gcp

In [4]:
import pandas as pd
tracking_df = pd.read_csv(os.path.join(upload_dir, 'upload_sheet.csv'))
tracking_df

,Order ID,Collection,Specimen,Corrected specimen name,Image type,Imaging type,Filename,ROI,Notes,Box URL,Filepath,Location,Upload to DCC
0,7,HT056P1,S1,NaN,JPG,NaN,IMG_0005.JPG,NaN,NaN,https://wustl.box.com/s/dbmeovn6zn677gpxezpmtk...,NaN,Box,False
1,8,HT056P1,S1,NaN,JPG,NaN,IMG_0006.JPG,NaN,NaN,https://wustl.box.com/s/dw4u7ov4u54bl75wy6fiw8...,NaN,Box,False
2,9,HT056P1,S1,NaN,JPG,NaN,IMG_0007.JPG,NaN,NaN,https://wustl.box.com/s/qkd7z5e2xrjs2ps41dncy0...,NaN,Box,False
3,10,HT056P1,S1,NaN,JPG,NaN,IMG_0008.JPG,NaN,NaN,https://wustl.box.com/s/i4zavkm1pl59rx9jvoufwr...,NaN,Box,False
4,6,HT056P1,S1,NaN,MOV,NaN,IMG_0004.MOV,NaN,NaN,https://wustl.box.com/s/sdhf4g2aw88rf4svwn4n53...,NaN,Box,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
183,168,HT168P1,S1H4L1 S1H4L4,NaN,TIFF,NaN,HT168P1_S1H4_091420_ROI_001.ome.tiff,1,done second,NaN,NaN,Storage1,False
184,169,HT168P1,S1H4L1 S1H4L4,NaN,TIFF,NaN,HT168P1_S1H4_091420_ROI_002.ome.tiff,2,done second,NaN,NaN,Storage1,False
185,160,HT168P1,S1H3L1 S1H3L4,"HT168P1-S1H3L1,S1H3L4",TIFF,IMC,HT168P1_S1H3_L1_L4_ROI_001.ome.tiff,1,added 2/23/2021,NaN,/storage1/fs1/dinglab/Active/Primary/HTAN.Imag...,Storage1,True
186,161,HT168P1,S1H3L1 S1H3L4,"HT168P1-S1H3L1,S1H3L4",TIFF,IMC,HT168P1_S1H3_L1_L4_ROI_002.ome.tiff,2,added 2/23/2021,NaN,/storage1/fs1/dinglab/Active/Primary/HTAN.Imag...,Storage1,True


In [7]:
list(tracking_df['Filename'])

['IMG_0005.JPG',
 'IMG_0006.JPG',
 'IMG_0007.JPG',
 'IMG_0008.JPG',
 'IMG_0004.MOV',
 'HT056P1 P.svs',
 'HT056P1 PB.svs',
 'HT56P B.svs',
 'HT56P E.svs',
 'HT56P1 A.svs',
 'HT56P1 C.svs',
 'HT56P1 D.svs',
 'HT060P P1.svs',
 'HT060P PB.svs',
 'HT60P1 REMAINS.svs',
 'HT61P A.svs',
 'HT61P B .svs',
 'HT61P C.svs',
 'HT64P1 H1.svs',
 'HT64P1 H3.svs',
 'HT64P1 H7.svs',
 'HT64P1 H9.svs',
 'HT071A H1.svs',
 'HT071A H4.svs',
 'HT071A H5*.svs',
 'HT071B H9.svs',
 'HT085P1 H1.svs',
 'HT085P1 H2.svs',
 'HT085P1 H3.svs',
 'HT085P1 H4.svs',
 'HT115P1 H3.svs',
 'HT115P1 H5.svs',
 'HT115P1 H6.svs',
 'HT121P1 H3.svs',
 'HT121P1 H4.svs',
 'HT121P1 H5.svs',
 'HT121P1 H9 A1.svs',
 'HT 122 P1 S1H3 Vp1 U11.svs',
 'HT 122 P1 S1H4 Vp1 U4.svs',
 'HT 122 P1 S1H5 Vp1 U22.svs',
 'HT 122 P1 S1H9 Vp1 U15.svs',
 'HT 123 P1 S1H1 1.svs',
 'HT 123 P1 S1H3 Vp1 U4.svs',
 'HT 123 P1 S1H5 Vp1 U4.svs',
 'HT 124 P1 S1H1 Vp1 U3.svs',
 'HT 124 P1 S1H2 Vp1 U14.svs',
 'HT 124 P1 S1H4 Vp1 U4.svs',
 'HT125P1 H3.svs',
 'HT125P1 H4

In [8]:
# replace filepath with new location
directory = '/storage1/fs1/dinglab/Active/Projects/estorrs/imaging/uploads/10072021/data/flattened'
tracking_df['Filepath'] = [f'{directory}/{fname}' for fname in tracking_df['Filename']]
tracking_df

,Order ID,Collection,Specimen,Corrected specimen name,Image type,Imaging type,Filename,ROI,Notes,Box URL,Filepath,Location,Upload to DCC
0,7,HT056P1,S1,NaN,JPG,NaN,IMG_0005.JPG,NaN,NaN,https://wustl.box.com/s/dbmeovn6zn677gpxezpmtk...,/storage1/fs1/dinglab/Active/Projects/estorrs/...,Box,False
1,8,HT056P1,S1,NaN,JPG,NaN,IMG_0006.JPG,NaN,NaN,https://wustl.box.com/s/dw4u7ov4u54bl75wy6fiw8...,/storage1/fs1/dinglab/Active/Projects/estorrs/...,Box,False
2,9,HT056P1,S1,NaN,JPG,NaN,IMG_0007.JPG,NaN,NaN,https://wustl.box.com/s/qkd7z5e2xrjs2ps41dncy0...,/storage1/fs1/dinglab/Active/Projects/estorrs/...,Box,False
3,10,HT056P1,S1,NaN,JPG,NaN,IMG_0008.JPG,NaN,NaN,https://wustl.box.com/s/i4zavkm1pl59rx9jvoufwr...,/storage1/fs1/dinglab/Active/Projects/estorrs/...,Box,False
4,6,HT056P1,S1,NaN,MOV,NaN,IMG_0004.MOV,NaN,NaN,https://wustl.box.com/s/sdhf4g2aw88rf4svwn4n53...,/storage1/fs1/dinglab/Active/Projects/estorrs/...,Box,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
183,168,HT168P1,S1H4L1 S1H4L4,NaN,TIFF,NaN,HT168P1_S1H4_091420_ROI_001.ome.tiff,1,done second,NaN,/storage1/fs1/dinglab/Active/Projects/estorrs/...,Storage1,False
184,169,HT168P1,S1H4L1 S1H4L4,NaN,TIFF,NaN,HT168P1_S1H4_091420_ROI_002.ome.tiff,2,done second,NaN,/storage1/fs1/dinglab/Active/Projects/estorrs/...,Storage1,False
185,160,HT168P1,S1H3L1 S1H3L4,"HT168P1-S1H3L1,S1H3L4",TIFF,IMC,HT168P1_S1H3_L1_L4_ROI_001.ome.tiff,1,added 2/23/2021,NaN,/storage1/fs1/dinglab/Active/Projects/estorrs/...,Storage1,True
186,161,HT168P1,S1H3L1 S1H3L4,"HT168P1-S1H3L1,S1H3L4",TIFF,IMC,HT168P1_S1H3_L1_L4_ROI_002.ome.tiff,2,added 2/23/2021,NaN,/storage1/fs1/dinglab/Active/Projects/estorrs/...,Storage1,True


In [9]:
tracking_df = tracking_df[tracking_df['Image type']=='TIFF']
tracking_df = tracking_df[tracking_df['Upload to DCC']]
tracking_df

,Order ID,Collection,Specimen,Corrected specimen name,Image type,Imaging type,Filename,ROI,Notes,Box URL,Filepath,Location,Upload to DCC
82,18,HT056P1,PAA1 PAA4,"HT056P1-PAA1,PAA4",TIFF,IMC,HT056P1_PA_A1_A4_ROI_01.ome.tiff,1,NaN,NaN,/storage1/fs1/dinglab/Active/Projects/estorrs/...,Storage1,True
83,19,HT056P1,PAA1 PAA4,"HT056P1-PAA1,PAA4",TIFF,IMC,HT056P1_PA_A1_A4_ROI_02.ome.tiff,2,NaN,NaN,/storage1/fs1/dinglab/Active/Projects/estorrs/...,Storage1,True
84,20,HT056P1,PAA1 PAA4,"HT056P1-PAA1,PAA4",TIFF,IMC,HT056P1_PA_A1_A4_ROI_03.ome.tiff,3,NaN,NaN,/storage1/fs1/dinglab/Active/Projects/estorrs/...,Storage1,True
85,21,HT056P1,PBA1 PBA4,"HT056P1-PBA1,PBA4",TIFF,IMC,HT056P1_PB_A1_A4_ROI_01.ome.tiff,1,NaN,NaN,/storage1/fs1/dinglab/Active/Projects/estorrs/...,Storage1,True
86,22,HT056P1,PBA1 PBA4,"HT056P1-PBA1,PBA4",TIFF,IMC,HT056P1_PB_A1_A4_ROI_02.ome.tiff,2,NaN,NaN,/storage1/fs1/dinglab/Active/Projects/estorrs/...,Storage1,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,158,HT168P1,S1H4L1 S1H4L4,"HT168P1-S1H4L1,S1H4L4",TIFF,IMC,HTAN_HT168P1_S1H4_L1_L4_ROI_002.ome.tiff,2,done first,NaN,/storage1/fs1/dinglab/Active/Projects/estorrs/...,Storage1,True
174,159,HT168P1,S1H4L1 S1H4L4,"HT168P1-S1H4L1,S1H4L4",TIFF,IMC,HTAN_HT168P1_S1H4_L1_L4_ROI_003.ome.tiff,3,done first,NaN,/storage1/fs1/dinglab/Active/Projects/estorrs/...,Storage1,True
185,160,HT168P1,S1H3L1 S1H3L4,"HT168P1-S1H3L1,S1H3L4",TIFF,IMC,HT168P1_S1H3_L1_L4_ROI_001.ome.tiff,1,added 2/23/2021,NaN,/storage1/fs1/dinglab/Active/Projects/estorrs/...,Storage1,True
186,161,HT168P1,S1H3L1 S1H3L4,"HT168P1-S1H3L1,S1H3L4",TIFF,IMC,HT168P1_S1H3_L1_L4_ROI_002.ome.tiff,2,added 2/23/2021,NaN,/storage1/fs1/dinglab/Active/Projects/estorrs/...,Storage1,True


In [10]:
# gsutil cp <file> gs://MyBucket/MyFolder/

In [11]:
f = open(os.path.join(upload_dir, 'upload_script.sh'), 'w')
for fp in tracking_df['Filepath']:
    s = fp.split('/')[-1]
    f.write(f'gsutil cp {fp} gs://htan-dcc-washu/imc_level_2/batch_1_10072021/{s}\n')
    # for csv too
    fp = fp.replace('.ome.tiff', '.csv')
    s = fp.split('/')[-1]
    f.write(f'gsutil cp {fp} gs://htan-dcc-washu/imc_level_2/batch_1_10072021/{s}\n')
f.close()

In [34]:
prev = open('../uploads/02262021/level2/upload_script.sh')
for line in prev:
    print(line)
prev.close()

gsutil cp /storage1/fs1/dinglab/Active/Primary/HTAN.Imaging/hyperion/02192021/b1_01042020/ome-tiff_htan/HT056P1_PA_A1_A4_ROI_01.ome.tiff gs://htan-dcc-washu/imc_level_2/batch_1_02242021/HT056P1_PA_A1_A4_ROI_01.ome.tiff

gsutil cp /storage1/fs1/dinglab/Active/Primary/HTAN.Imaging/hyperion/02192021/b1_01042020/ome-tiff_htan/HT056P1_PA_A1_A4_ROI_02.ome.tiff gs://htan-dcc-washu/imc_level_2/batch_1_02242021/HT056P1_PA_A1_A4_ROI_02.ome.tiff

gsutil cp /storage1/fs1/dinglab/Active/Primary/HTAN.Imaging/hyperion/02192021/b1_01042020/ome-tiff_htan/HT056P1_PA_A1_A4_ROI_03.ome.tiff gs://htan-dcc-washu/imc_level_2/batch_1_02242021/HT056P1_PA_A1_A4_ROI_03.ome.tiff

gsutil cp /storage1/fs1/dinglab/Active/Primary/HTAN.Imaging/hyperion/02192021/b1_01042020/ome-tiff_htan/HT056P1_PB_A1_A4_ROI_01.ome.tiff gs://htan-dcc-washu/imc_level_2/batch_1_02242021/HT056P1_PB_A1_A4_ROI_01.ome.tiff

gsutil cp /storage1/fs1/dinglab/Active/Primary/HTAN.Imaging/hyperion/02192021/b1_01042020/ome-tiff_htan/HT056P1_PB_A1_A4

In [32]:
!wc -l ../uploads/02262021/level2/upload_sheet.sh

wc: ../uploads/02262021/level2/upload_sheet.sh: No such file or directory
